# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
g_key = ""

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = pd.read_csv("city_weather.csv")

cities = cities.drop(columns=["Unnamed: 0"])
cities.head()

,City,Cloudiness,Country,Date,Humidity,Temp,Lat,Lng,Wind Speed
0,victoria,67,HK,1607381085,69,62.64,22.29,114.16,12.19
1,mataura,1,NZ,1607381085,60,66.99,-46.19,168.86,8.01
2,tyukhtet,100,RU,1607381085,93,9.79,56.54,89.30,15.82
3,clyde river,90,CA,1607381085,84,8.60,70.47,-68.59,3.36
4,golden gate,1,US,1607381085,82,65.98,26.19,-81.70,12.75


In [3]:
cities.count()

City          547
Cloudiness    547
Country       542
Date          547
Humidity      547
Temp          547
Lat           547
Lng           547
Wind Speed    547
dtype: int64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key="")

# Store lat and lon in locations
locations = cities[["Lat", "Lng"]]
weight = cities["Humidity"].astype(float)

In [5]:
# Plot heatmap
fig = gmaps.figure()

# Create heatmap layer
heatmap_layer = gmaps.heatmap_layer(locations, weights=weight,
                                   dissipating=False, max_intensity=10,
                                   point_radius=1)
# Add layer to heatmap
fig.add_layer(heatmap_layer)

# Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. 
# You want to be sure the weather is ideal

ideal_city = cities.loc[(cities["Temp"] > 70) & (cities["Temp"] < 80) & 
                        (cities["Wind Speed"] < 10) &
                        (cities["Cloudiness"] == 0),
                        :]
ideal_city = ideal_city.dropna(how='any')
ideal_city.reset_index(inplace=True)
ideal_city






# #------------------------------------------------------------------------------------------------------
# new_types_df = pd.DataFrame(weather_data_df, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])

# max_temp = (new_types_df["Max Temp"] <= 80) & (new_types_df["Max Temp"] > 70)
# wind_speed = new_types_df["Wind Speed"] < 10
# cloudiness = new_types_df["Cloudiness"] == 0

# # new_types_df[max_temp & wind_speed & cloudiness]
# new_types_df[max_temp & wind_speed & cloudiness]

,index,City,Cloudiness,Country,Date,Humidity,Temp,Lat,Lng,Wind Speed
0,85,puerto ayora,0,EC,1607381094,61,77.77,-0.74,-90.35,8.05
1,114,coihaique,0,CL,1607381096,27,75.20,-45.58,-72.07,8.05
2,159,vallenar,0,CL,1607381101,38,73.26,-28.57,-70.76,6.87
3,220,itapora,0,BR,1607381116,81,74.48,-22.08,-54.79,3.96
4,243,mtimbira,0,TZ,1607381118,76,74.98,-8.78,36.35,2.82
5,244,umm lajj,0,SA,1607381118,32,73.72,25.02,37.27,5.30
6,251,santiago,0,CL,1607380911,31,75.45,-33.46,-70.65,8.05
7,260,salalah,0,OM,1607381120,73,73.40,17.02,54.09,2.24
8,297,guerrero negro,0,MX,1607381124,19,75.76,27.98,-114.06,7.67
9,327,machali,0,CL,1607381060,36,78.80,-34.18,-70.67,6.11


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
hotel_data = ideal_city.loc[:,["City", "Country", "Lat", "Lng"]]

hotel_data["Hotel Name"] = ""

hotel_data

,City,Country,Lat,Lng,Hotel Name
0,puerto ayora,EC,-0.74,-90.35,
1,coihaique,CL,-45.58,-72.07,
2,vallenar,CL,-28.57,-70.76,
3,itapora,BR,-22.08,-54.79,
4,mtimbira,TZ,-8.78,36.35,
5,umm lajj,SA,25.02,37.27,
6,santiago,CL,-33.46,-70.65,
7,salalah,OM,17.02,54.09,
8,guerrero negro,MX,27.98,-114.06,
9,machali,CL,-34.18,-70.67,


In [13]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Create parameter dict
params = {"type": "hotel",
            "keyword": "hotel",
            "radius": 5000, 
            "key": g_key}

In [16]:
# Loop through lat, lng, city name
for index, row in hotel_data.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
   
    # add location to parameter dict
    params["location"] = f"{lat},{lng}"
    
    # call url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_data.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 10 sec to make another api request to avoid SSL Error
    time.sleep(10)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 0: puerto ayora.
Closest hotel in puerto ayora is La Isla Hotel.
------------
Retrieving Results for Index 1: coihaique.
Closest hotel in coihaique is Entre Cumbres Apart Hotel.
------------
Retrieving Results for Index 2: vallenar.
Closest hotel in vallenar is Cabañas Del Frances.
------------
Retrieving Results for Index 3: itapora.
Closest hotel in itapora is Hotel Santa Fé.
------------
Retrieving Results for Index 4: mtimbira.
Missing field/result... skipping.
------------
Retrieving Results for Index 5: umm lajj.
Closest hotel in umm lajj is HP Red Sea Hotel.
------------
Retrieving Results for Index 6: santiago.
Closest hotel in santiago is Hotel Brasilia.
------------
Retrieving Results for Index 7: salalah.
Closest hotel in salalah is Al Dyafa Hotel Suites.
------------
Retrieving Results for Index 8: guerrero negro.
Closest hotel in guerrero negro is The Halfway Inn.
------------
Retrieving Results for Index 9: machali.
Closest hotel in machali is

In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_data.iterrows()]
locations = hotel_data[["Lat", "Lng"]]

In [19]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))